In [41]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [42]:


def single_game_player_df(week, year, day, file_name,
                         qb_file, rb_file, wr_file,
                         te_file, def_file,
                         team1,
                         team2):

    fanduel = pd.read_csv(file_name)
    fanduel = fanduel[fanduel['keep']=='x']
    fanduel = fanduel[(fanduel['Injury Indicator'] != 'IR') &
                  (fanduel['Injury Indicator'] != 'O') &
                  (fanduel['Injury Indicator'] != 'D')]
    fanduel2 = fanduel[['Player ID + Player Name', 'Nickname','Position', 'Team',
                    'Salary', 'Injury Indicator','FPPG','Tier']]
    fanduel2 = fanduel2[fanduel2['Position'] != 'D']
    fanduel2['player_wk_year'] = fanduel2['Nickname'] + " " + week + " " + year

    fanduel3 = fanduel[['Player ID + Player Name', 'Nickname','Position', 'Team',
                    'Salary', 'Injury Indicator','FPPG','Tier']]
    fanduel3 = fanduel3[fanduel3['Position'] == 'D']
    fanduel3['player_wk_year'] = fanduel3['Team'] + " " + week + " " + year

    fanduel4 = fanduel2.append(fanduel3)
    fanduel4 = fanduel4[['Player ID + Player Name', 'Position', 
                     'Team','Salary', 'Injury Indicator',
                    'player_wk_year','FPPG','Tier']]
    #fanduel4.sort_values(by = 'Salary', ascending = False)

    fanduel4['pos_rank'] = fanduel4.groupby(['Team','Position'])['Salary'].rank(ascending = False).astype(int)
    fanduel4['pos_rank'] = fanduel4['Position'] + fanduel4['pos_rank'].astype(str)

    fanduel5 = fanduel4[fanduel4['pos_rank'].isin(['QB1','RB1','RB2','RB3','WR1',
                                               'WR2','WR3','WR4','WR5',
                                              'TE1','TE2','TE3','D1','K1'])]

    qb_df = pd.read_csv(qb_file)
    qb_df['position'] = 'qb'
    rb_df = pd.read_csv(rb_file)
    rb_df['position'] = 'rb'
    wr_df = pd.read_csv(wr_file)
    wr_df['position'] = 'wr'
    te_df = pd.read_csv(te_file)
    te_df['position'] = 'te'
    def_df = pd.read_csv(def_file)
    def_df['position'] = 'def'
    def_df = def_df.rename(columns = {'team_wk_year':'player_wk_year'})

    df = qb_df.append(rb_df)
    df = df.append(wr_df)
    df = df.append(te_df)
    df = df.append(def_df)

    df = df.reset_index()
    df = df.drop(columns = ['Unnamed: 0', 'index'])

    df = df[(df['team_wk_year'].str.contains(team1)) |
            (df['team_wk_year'].str.contains(team2))]
    df_final = pd.merge(fanduel5, df, how = 'left', 
                    on = 'player_wk_year', suffixes = (None, None))

    #df_final['Prob2'] = df_final[['FPPG', 'Prob']].mean(skipna=True, axis = 1)
    #df_final['Prob2'] = df_final['FPPG']
    df_final['Prob2'] = df_final['Tier']
    df_final = df_final.drop_duplicates(subset = 'player_wk_year', keep = 'last')
    df_final = df_final.sort_values(by = 'Prob2', ascending = False)

    return df_final

In [82]:
#team1 = fav
#team2 = underdog
df_final = single_game_player_df(week = '9',
                        year = '2022',
                        day = 'THu',
                        file_name = '2022week9_thu_night_fanduel_players.csv',
                        qb_file = 'week9_scored_qb.csv',
                        rb_file = 'week9_scored_rb.csv',
                        wr_file = 'week9_scored_wr.csv',
                        te_file = 'week9_scored_te.csv',
                        def_file = 'week9_scored_def.csv',
                        team1 = 'PHI',
                        team2 = 'HOU')

In [83]:
df_final = df_final[['Player ID + Player Name','Position', 'Team', 'Salary',
                    'player_wk_year','Prob2','Tier']]#,'FPPG','Prob']]

In [84]:
df_final

,Player ID + Player Name,Position,Team,Salary,player_wk_year,Prob2,Tier
0,82605-69531:Jalen Hurts,QB,PHI,17500,Jalen Hurts 9 2022,5.0,5.0
3,82605-72846:A.J. Brown,WR,PHI,14500,A.J. Brown 9 2022,5.0,5.0
5,82605-71845:Miles Sanders,RB,PHI,13000,Miles Sanders 9 2022,5.0,5.0
1,82605-86055:Dameon Pierce,RB,HOU,15000,Dameon Pierce 9 2022,5.0,5.0
11,82605-83074:Dallas Goedert,TE,PHI,11000,Dallas Goedert 9 2022,5.0,5.0
25,82605-33047:Chris Moore,WR,HOU,6500,Chris Moore 9 2022,4.0,4.0
15,82605-25813:Phillip Dorsett II,WR,HOU,9000,Phillip Dorsett II 9 2022,3.0,3.0
38,82605-12545:Philadelphia Eagles,D,PHI,10000,PHI 9 2022,3.0,3.0
17,82605-90912:Kenneth Gainwell,RB,PHI,8000,Kenneth Gainwell 9 2022,3.0,3.0
4,82605-89628:Davis Mills,QB,HOU,14000,Davis Mills 9 2022,3.0,3.0


In [65]:
def create_single_game_simulations(df_final, team1, team2, 
                                   team1_pos, team2_pos
                                  # outcomes
                                  ):#,
                                 # players1, players2,
                                 # players3, players4):
    print('team1:')
    print(team1)
    print('team2:')
    print(team2)
    result = pd.DataFrame() 
    team1_df = df_final[df_final['Team'] == team1]
    team1_df['pos_rank'] = team1_df.groupby('Position')['Salary'].rank(ascending = False).astype(int)
    team1_df['pos_rank'] = team1_df['Position'] + team1_df['pos_rank'].astype(str)
    team2_df = df_final[df_final['Team'] == team2]
    team2_df['pos_rank'] = team2_df.groupby('Position')['Salary'].rank(ascending = False).astype(int)
    team2_df['pos_rank'] = team2_df['Position'] + team2_df['pos_rank'].astype(str)
    
    
    #outcomes = ['HL',
    #            'HM',
    #            'HH',
    #            'ML',
    #            'MM',
    #            'MH'#,
                #'LL',
                #'LM',
                #'LH'
     #          ]
    
    #for outcome in outcomes:
    #    print(outcome)
    #    if (outcome == 'HL'):
    #        team1_df2 = team1_df[team1_df['pos_rank'].isin(['QB1','RB1','RB2','WR1','WR2','D1','K1'])]
    #        team2_df2 = team2_df[team2_df['pos_rank'].isin(['K1','QB1','RB1','WR1'])]
    #    elif (outcome == 'HM'):
    #        team1_df2 = team1_df[team1_df['pos_rank'].isin(['QB1','RB1','RB2','WR1','WR2','WR3',
    #                                                        'TE1','D1'])]
    #        team2_df2 = team2_df[team2_df['pos_rank'].isin(['QB1','RB1','WR1','K1'])]
    #    elif (outcome == 'HH'):
    #        team1_df2 = team1_df[team1_df['pos_rank'].isin(['QB1','RB1','RB2','WR1','WR2','WR3',
    #                                                        'WR4','WR5','TE1','TE2','D1'])]
    #        team2_df2 = team2_df[team2_df['pos_rank'].isin(['QB1','RB1','WR1','WR2','WR3','TE1'
    #                                                    ])]
    #    elif (outcome == 'ML'):
    #        team1_df2 = team1_df[team1_df['pos_rank'].isin(['QB1','RB1','RB2','WR1','WR2','D1','K1'])]
    #        team2_df2 = team2_df[team2_df['pos_rank'].isin(['QB1','RB1','WR1','WR2','K1'])]
    #    elif (outcome == 'MM'):
    #        team1_df2 = team1_df[team1_df['pos_rank'].isin(['QB1','RB1','RB2','WR1','WR2','TE1',
    #                                                        'WR3','D1'])]
    #        team2_df2 = team2_df[team2_df['pos_rank'].isin(['QB1','RB1','WR1','WR2'])]
    #    elif (outcome == 'MH'):
    #        team1_df2 = team1_df[team1_df['pos_rank'].isin(['QB1','RB1','RB2','TE1','WR1','WR2',
    #                                                       'WR3','WR4','WR5'])]
    #        team2_df2 = team2_df[team2_df['pos_rank'].isin(['QB1','RB1','WR1','WR2','WR3'])]
    #    elif (outcome == 'LL'):
    #        team1_df2 = team1_df[team1_df['pos_rank'].isin(['D1','K1','RB1','QB1','RB2','WR1','WR2','TE1'])]
    #        team2_df2 = team2_df[team2_df['pos_rank'].isin(['D1','K1','RB1','QB1','RB2','WR1','WR2','TE1'])]
    #    elif (outcome == 'LM'):
    #        team1_df2 = team1_df[team1_df['pos_rank'].isin(['K1','D1','QB1','RB1','RB2',
    #                                                       'WR1','WR2','WR3'])]
    #        team2_df2 = team2_df[team2_df['pos_rank'].isin(['K1','D1','QB1','RB1','RB2',
    #                                                       'WR1','WR2','WR3'])]
    #    elif (outcome == 'LH'):
    #        team1_df2 = team1_df[team1_df['pos_rank'].isin(['K1','D1','QB1','RB1','RB2','TE1','TE2',
    #                                                       'WR1','WR2','WR3','WR4','WR5'])]
    #        team2_df2 = team2_df[team2_df['pos_rank'].isin(['K1','D1','QB1','RB1','RB2','TE1', 'TE2',
    #                                                       'WR1','WR2','WR3','WR4','WR5'])]
 
    team1_df2 = team1_df[team1_df['pos_rank'].isin(team1_pos)]
    team2_df2 = team2_df[team2_df['pos_rank'].isin(team2_pos)]
    
    team1_team2 = team1_df2.append(team2_df2)
    df_final_qb = team1_team2.sort_values('Prob2',ascending= False)
    df1 = df_final_qb[['Player ID + Player Name','Team', 'Salary', 'Prob2']]
    df1 = df1.rename(columns = {'Player ID + Player Name':'flex1',
                                             'Salary':'flex1_sal','Prob2':'flex1_prob'})
    df2 = df_final_qb[['Player ID + Player Name', 'Team','Salary', 'Prob2']]
    df2 = df2.rename(columns = {'Player ID + Player Name':'flex2',
                                             'Salary':'flex2_sal','Prob2':'flex2_prob'})
    df3 = df_final_qb[['Player ID + Player Name','Team', 'Salary', 'Prob2']]
    df3 = df3.rename(columns = {'Player ID + Player Name':'flex3',
                                            'Salary':'flex3_sal','Prob2':'flex3_prob'})
    df4 = df_final_qb[['Player ID + Player Name','Team', 'Salary', 'Prob2']]
    df4 = df4.rename(columns = {'Player ID + Player Name':'flex4',
                                             'Salary':'flex4_sal','Prob2':'flex4_prob'})
    df5 = df_final_qb[['Player ID + Player Name','Team', 'Salary', 'Prob2']]
    df5 = df5.rename(columns = {'Player ID + Player Name':'flex5',
                                             'Salary':'flex5_sal','Prob2':'flex5_prob'})

    sim_df = pd.merge(df1, df2, how = 'cross')
    sim_df = pd.merge(sim_df, df3, how = 'cross')
    sim_df = pd.merge(sim_df, df4, how = 'cross')
    sim_df = pd.merge(sim_df, df5, how = 'cross')
        
        
    sim_df = sim_df[(sim_df['flex1'] != sim_df['flex2']) & (sim_df['flex1'] != sim_df['flex3']) &
                (sim_df['flex1'] != sim_df['flex4']) & (sim_df['flex1'] != sim_df['flex5']) &
                (sim_df['flex2'] != sim_df['flex3']) & (sim_df['flex2'] != sim_df['flex4']) &
                (sim_df['flex2'] != sim_df['flex5']) & (sim_df['flex3'] != sim_df['flex4']) &
                (sim_df['flex3'] != sim_df['flex5']) & (sim_df['flex4'] != sim_df['flex5'])]

    sim_df['total_salary'] = sim_df['flex1_sal'] + sim_df['flex2_sal'] + sim_df['flex3_sal'] + sim_df['flex4_sal'] + sim_df['flex5_sal']
    sim_df['total_proj'] = sim_df['flex1_prob'] + sim_df['flex2_prob'] + sim_df['flex3_prob'] + sim_df['flex4_prob'] + sim_df['flex5_prob']

    sim_df = sim_df[sim_df['total_salary'] <= 60000]
    sim_df = sim_df[sim_df['total_salary'] > 57000]
    #if (len(sim_df) == 0):
    #    continue

    final= sim_df[['flex1', 'flex2', 'flex3', 'flex4', 'flex5','total_proj']]
    final['total_proj'] = final['total_proj'].astype(str)
        #final['total_salary'] = final['total_salary'].astype(str)
    print(len(final))
    n = len(final)
       # if (len(final) < 10000):
       #     n = len(final)
       # else:
       #     n = 10000
    final2 = pd.DataFrame()
    for i in range(0,n):
          #  print(i)
        l = final.iloc[i,:].values.tolist()
            #print(l)
        l.sort()
           # if ((players1 in l) |
           #     (players2 in l) |
           #    (players3 in l) |
           #     (players4 in l)
           #    ):
               # print(l)
        temp = pd.DataFrame([l])
            
        final2 = final2.append(temp)
            
    final2 = final2.drop_duplicates(subset = [1,2,3,4,5], keep = 'first')
        
    df_team_check = df_final_qb[['Player ID + Player Name','Team']]
        
    for i in range(1,6):
        print(i)
        final2 = pd.merge(final2, df_team_check, how = 'inner', left_on = i,
                          right_on = 'Player ID + Player Name',
                          suffixes = (None,None))
        team_col = 'Team' + str(i)
        final2[team_col] = final2['Team']
        final2 = final2.drop(columns = ['Team','Player ID + Player Name'])
    keep_list = []
    team1_v_team2 = []
    for i in range(0,len(final2)):
        team1_count = 0
        team2_count = 0
            #keep_list = []
        teams = final2[['Team1','Team2','Team3','Team4','Team5']]
        l = teams.iloc[i,:].values.tolist()
            #print(l)
        for j in l:
             #   print(j)
                
            if (j == team1):
                team1_count = team1_count + 1
            elif (j == team2):
                team2_count = team2_count + 1
             #   print(team1_count)
             #   print(team2_count)
        if ((team1_count > 4) | (team2_count > 4)):
            keep = 'No'
        else:
            keep = 'Yes'
                
        if ((team1_count == 2) & (team2_count == 3)):
            team1_team2 = '2:3'
        elif ((team1_count == 3) & (team2_count == 2)):
            team1_team2 = '3:2'
        elif ((team1_count == 4) & (team2_count == 1)):
            team1_team2 = '4:1'
        elif ((team1_count == 1) & (team2_count == 4)):
            team1_team2 = '1:4'
        else:
            team1_team2 = 'none'
                
          #  print(keep)
        keep_list.append(keep)
        team1_v_team2.append(team1_team2)
            #print(keep_list)
    final2['keep'] = keep_list
    final2['team1_team2_count'] = team1_v_team2
               
       # if(len(final2) < 33):
       #     m = len(final2)
       # else: 
       #     m = 33
       # final2 = final2.head(m)
    result = result.append(final2)
    
    result = result[result['keep'] == 'Yes']
    #result = result[[0,1,2,3,4,5]]
    result = result.drop_duplicates(subset = [1,2,3,4,5], keep = 'first')

    
    return result

In [67]:
#fav: KC
#spread: -7
#o/u: 51.5
#team1 =favorite
#team2 =underdog

test = create_single_game_simulations(df_final = df_final,
                                      team1 = 'PHI',
                                      team2 = 'HOU',
                                      team1_pos = ['QB1','RB1','RB2','TE1','TE2',
                                                  'WR1','WR2','WR3',
                                                   'K1','D1'],
                                      team2_pos = ['QB1','RB1','RB2','TE1','TE2',
                                                   'WR1','WR2','WR3',
                                                   'K1','D1'])
                                      #outcomes = ['LH']#,
                                   #  players1 = '81393-22015:Russell Wilson',
                                   #  players2 = '81393-69213:Michael Pittman Jr.',
                                   #  players3 = '81393-6616:Matt Ryan',
                                  # #  players4 = "81556-85701:Ja'Marr Chase"
                                   #  )


team1:
PHI
team2:
HOU
155160
1
2
3
4
5


In [68]:
test.shape

(1250, 13)

In [69]:
test = test.drop_duplicates(subset = [1,2,3,4,5], keep = 'first')


In [70]:
test.sort_values(0, ascending = False).head(50)

,0,1,2,3,4,5,Team1,Team2,Team3,Team4,Team5,keep,team1_team2_count
872,22.0,82605-29501:Brandin Cooks,82605-41235:Jake Elliott,82605-71845:Miles Sanders,82605-72846:A.J. Brown,82605-83074:Dallas Goedert,HOU,PHI,PHI,PHI,PHI,Yes,4:1
181,22.0,82605-12545:Philadelphia Eagles,82605-41235:Jake Elliott,82605-72846:A.J. Brown,82605-83074:Dallas Goedert,82605-86055:Dameon Pierce,PHI,PHI,PHI,PHI,HOU,Yes,4:1
833,22.0,82605-29932:Ka'imi Fairbairn,82605-41235:Jake Elliott,82605-69531:Jalen Hurts,82605-71845:Miles Sanders,82605-83074:Dallas Goedert,HOU,PHI,PHI,PHI,PHI,Yes,4:1
199,22.0,82605-12545:Philadelphia Eagles,82605-41235:Jake Elliott,82605-71845:Miles Sanders,82605-83074:Dallas Goedert,82605-86055:Dameon Pierce,PHI,PHI,PHI,PHI,HOU,Yes,4:1
571,22.0,82605-29932:Ka'imi Fairbairn,82605-41235:Jake Elliott,82605-71845:Miles Sanders,82605-72846:A.J. Brown,82605-90576:DeVonta Smith,HOU,PHI,PHI,PHI,PHI,Yes,4:1
665,22.0,82605-12545:Philadelphia Eagles,82605-41235:Jake Elliott,82605-71845:Miles Sanders,82605-86055:Dameon Pierce,82605-90576:DeVonta Smith,PHI,PHI,PHI,HOU,PHI,Yes,4:1
26,22.0,82605-12545:Philadelphia Eagles,82605-29932:Ka'imi Fairbairn,82605-41235:Jake Elliott,82605-69531:Jalen Hurts,82605-72846:A.J. Brown,PHI,HOU,PHI,PHI,PHI,Yes,4:1
16,22.0,82605-12545:Philadelphia Eagles,82605-29501:Brandin Cooks,82605-41235:Jake Elliott,82605-71845:Miles Sanders,82605-72846:A.J. Brown,PHI,HOU,PHI,PHI,PHI,Yes,4:1
1257,22.0,82605-12545:Philadelphia Eagles,82605-29932:Ka'imi Fairbairn,82605-41235:Jake Elliott,82605-69531:Jalen Hurts,82605-71845:Miles Sanders,PHI,HOU,PHI,PHI,PHI,Yes,4:1
182,21.0,82605-29932:Ka'imi Fairbairn,82605-41235:Jake Elliott,82605-72846:A.J. Brown,82605-83074:Dallas Goedert,82605-86055:Dameon Pierce,HOU,PHI,PHI,PHI,HOU,Yes,3:2


In [11]:
#vegas odds:
#fav: DEN
#spread: -3
#o/u: 42


In [71]:
#team1 = BUF, so as heavy fav we will subset only 4:1 and 3:2
#fav should always get a '4:1'
team_ratio = ['4:1','3:2']#,'2:3','1:4']

In [72]:
test = test.sort_values(0, ascending = False)#.head(150)
test2 = test[test['team1_team2_count'].isin(team_ratio)]

In [73]:
test2#.to_csv('thu_night_wk13_lineups_test.csv')

,0,1,2,3,4,5,Team1,Team2,Team3,Team4,Team5,keep,team1_team2_count
872,22.0,82605-29501:Brandin Cooks,82605-41235:Jake Elliott,82605-71845:Miles Sanders,82605-72846:A.J. Brown,82605-83074:Dallas Goedert,HOU,PHI,PHI,PHI,PHI,Yes,4:1
181,22.0,82605-12545:Philadelphia Eagles,82605-41235:Jake Elliott,82605-72846:A.J. Brown,82605-83074:Dallas Goedert,82605-86055:Dameon Pierce,PHI,PHI,PHI,PHI,HOU,Yes,4:1
833,22.0,82605-29932:Ka'imi Fairbairn,82605-41235:Jake Elliott,82605-69531:Jalen Hurts,82605-71845:Miles Sanders,82605-83074:Dallas Goedert,HOU,PHI,PHI,PHI,PHI,Yes,4:1
199,22.0,82605-12545:Philadelphia Eagles,82605-41235:Jake Elliott,82605-71845:Miles Sanders,82605-83074:Dallas Goedert,82605-86055:Dameon Pierce,PHI,PHI,PHI,PHI,HOU,Yes,4:1
571,22.0,82605-29932:Ka'imi Fairbairn,82605-41235:Jake Elliott,82605-71845:Miles Sanders,82605-72846:A.J. Brown,82605-90576:DeVonta Smith,HOU,PHI,PHI,PHI,PHI,Yes,4:1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,13.0,82605-12556:Houston Texans,82605-25813:Phillip Dorsett II,82605-53390:Zach Pascal,82605-69531:Jalen Hurts,82605-72846:A.J. Brown,HOU,HOU,PHI,PHI,PHI,Yes,3:2
46,13.0,82605-25813:Phillip Dorsett II,82605-27338:Rex Burkhead,82605-53390:Zach Pascal,82605-69531:Jalen Hurts,82605-72846:A.J. Brown,HOU,HOU,PHI,PHI,PHI,Yes,3:2
941,13.0,82605-12556:Houston Texans,82605-53390:Zach Pascal,82605-69531:Jalen Hurts,82605-86055:Dameon Pierce,82605-90912:Kenneth Gainwell,HOU,PHI,PHI,HOU,PHI,Yes,3:2
940,13.0,82605-25813:Phillip Dorsett II,82605-53390:Zach Pascal,82605-69531:Jalen Hurts,82605-86055:Dameon Pierce,82605-90912:Kenneth Gainwell,HOU,PHI,PHI,HOU,PHI,Yes,3:2


In [74]:
df_final

,Player ID + Player Name,Position,Team,Salary,player_wk_year,Prob2,Tier
0,82605-69531:Jalen Hurts,QB,PHI,17500,Jalen Hurts 9 2022,5.0,5.0
3,82605-72846:A.J. Brown,WR,PHI,14500,A.J. Brown 9 2022,5.0,5.0
5,82605-71845:Miles Sanders,RB,PHI,13000,Miles Sanders 9 2022,5.0,5.0
12,82605-41235:Jake Elliott,K,PHI,9500,Jake Elliott 9 2022,5.0,5.0
1,82605-86055:Dameon Pierce,RB,HOU,15000,Dameon Pierce 9 2022,4.0,4.0
6,82605-90576:DeVonta Smith,WR,PHI,12000,DeVonta Smith 9 2022,4.0,4.0
11,82605-83074:Dallas Goedert,TE,PHI,11000,Dallas Goedert 9 2022,4.0,4.0
38,82605-12545:Philadelphia Eagles,D,PHI,10000,PHI 9 2022,4.0,4.0
4,82605-89628:Davis Mills,QB,HOU,14000,Davis Mills 9 2022,3.0,3.0
10,82605-29501:Brandin Cooks,WR,HOU,11500,Brandin Cooks 9 2022,3.0,3.0


In [75]:

def create_mvp_lineups(test_df,
                       df_final,
                       team_ratios,
                      counts,
                      mvps):

    test_df = pd.DataFrame()
    team1_team2 = team_ratios #['4:1','4:1','4:1' ,'4:1', '3:2','3:2','3:2','3:2']
    count = counts #[40,20,10,5,40,20,10,15]
    mvp = mvps #['84053-62239:Josh Allen','84053-25079:Stefon Diggs', '84053-12526:Buffalo Bills','84053-6642:Nick Folk',
      #'84053-62239:Josh Allen','84053-25079:Stefon Diggs',
      #'84053-111540:Rhamondre Stevenson','84053-90584:Mac Jones']
    n = len(team1_team2)

    for z in range(0,n):
    
        teams = team1_team2[z]
        counts = count[z]
        mvps_filter = mvp[z]
        print(teams)
        test_new = test2[test2['team1_team2_count'] == teams]#.head(counts)
        print(len(test_new))
        #x = 0
       # while(x < counts):
        j = 0
        for pos in range(0,len(test_new)):
        #j = 0
        #pos = 0
        #while ((j <= counts) or (pos < len(test_new)) ):
        
            df_final_mvp = df_final['Player ID + Player Name'][df_final['Player ID + Player Name'].isin([mvps_filter])].values.tolist()
            df_final_1 = df_final['Player ID + Player Name'].values.tolist()
            df_final_2 = df_final['Player ID + Player Name'].values.tolist()
            df_final_3 = df_final['Player ID + Player Name'].values.tolist()
            df_final_4 = df_final['Player ID + Player Name'].values.tolist()

            l = test_new.iloc[pos, :].values.tolist()


            l_mvp = []
            l_1 = []
            l_2 = []
            l_3 = []
            l_4 = []

            for i in range(1,6):
               # print(l[i])
                if ((l[i] in df_final_mvp) & (len(l_mvp) < 1)):
                    l_mvp = l[i]
                    df_final_mvp.remove(l[i])
                    df_final_1.remove(l[i])
                    df_final_2.remove(l[i])
                    df_final_3.remove(l[i])
                    df_final_4.remove(l[i])

                elif ((l[i] in (df_final_1)) & (len(l_1) < 1)):
                    l_1 = l[i]
                    df_final_1.remove(l[i])
                    df_final_2.remove(l[i])
                    df_final_3.remove(l[i])
                    df_final_4.remove(l[i])
                    
                elif ((l[i] in (df_final_2)) & (len(l_2) < 1)):
                    l_2 = l[i]
                    df_final_1.remove(l[i])
                    df_final_2.remove(l[i])
                    df_final_3.remove(l[i])
                    df_final_4.remove(l[i])
                elif ((l[i] in (df_final_3)) & (len(l_3) < 1)):
                    l_3 = l[i]
                    df_final_1.remove(l[i])
                    df_final_2.remove(l[i])
                    df_final_3.remove(l[i])
                    df_final_4.remove(l[i])
                elif ((l[i] in (df_final_4)) & (len(l_4) < 1)):
                    l_4 = l[i]
                    df_final_1.remove(l[i])
                    df_final_2.remove(l[i])
                    df_final_3.remove(l[i])
                    df_final_4.remove(l[i])
            
            final_l = [l_mvp, l_1, l_2, l_3, l_4]
            final_l = pd.DataFrame([final_l])
            if (len(l_mvp) > 2):
               # pos = pos + 1
                j = j + 1
            else:
              #  pos = pos + 1
                j = j
           # print(pos)
            #print(final_l)
            test_df = test_df.append(final_l)
            test_df = test_df[test_df[0].str.len() > 2]
            if (j == counts):
                break

            
    return test_df

In [76]:
df_final

,Player ID + Player Name,Position,Team,Salary,player_wk_year,Prob2,Tier
0,82605-69531:Jalen Hurts,QB,PHI,17500,Jalen Hurts 9 2022,5.0,5.0
3,82605-72846:A.J. Brown,WR,PHI,14500,A.J. Brown 9 2022,5.0,5.0
5,82605-71845:Miles Sanders,RB,PHI,13000,Miles Sanders 9 2022,5.0,5.0
12,82605-41235:Jake Elliott,K,PHI,9500,Jake Elliott 9 2022,5.0,5.0
1,82605-86055:Dameon Pierce,RB,HOU,15000,Dameon Pierce 9 2022,4.0,4.0
6,82605-90576:DeVonta Smith,WR,PHI,12000,DeVonta Smith 9 2022,4.0,4.0
11,82605-83074:Dallas Goedert,TE,PHI,11000,Dallas Goedert 9 2022,4.0,4.0
38,82605-12545:Philadelphia Eagles,D,PHI,10000,PHI 9 2022,4.0,4.0
4,82605-89628:Davis Mills,QB,HOU,14000,Davis Mills 9 2022,3.0,3.0
10,82605-29501:Brandin Cooks,WR,HOU,11500,Brandin Cooks 9 2022,3.0,3.0


In [79]:
team_ratios = [
               '4:1','4:1','4:1','4:1','4:1','4:1','4:1',
    
                '3:2','3:2','3:2','3:2','3:2','3:2','3:2'
    
              # '2:3','2:3','2:3','2:3',#'2:3',#'2:3'#,
    
               #'1:4','1:4','1:4','1:4'#,'1:4'
    
               
              ]
counts = [
    
    20,10,10,7,10,7,7,
    25,12,12,7,12,7,7
    
         ]
   
mvps = [
    '82605-69531:Jalen Hurts','82605-72846:A.J. Brown',
    '82605-71845:Miles Sanders','82605-41235:Jake Elliott',
    '82605-90576:DeVonta Smith','82605-83074:Dallas Goedert',
    '82605-12545:Philadelphia Eagles',

    '82605-69531:Jalen Hurts','82605-72846:A.J. Brown',
    '82605-71845:Miles Sanders','82605-41235:Jake Elliott',
    '82605-90576:DeVonta Smith','82605-83074:Dallas Goedert',
    '82605-12545:Philadelphia Eagles'


       ]


test_df = create_mvp_lineups(test_df = test2,
                       df_final = df_final,
                       team_ratios = team_ratios,
                      counts = counts,
                      mvps = mvps)

4:1
261
4:1
261
4:1
261
4:1
261
4:1
261
4:1
261
4:1
261
3:2
518
3:2
518
3:2
518
3:2
518
3:2
518
3:2
518
3:2
518


In [80]:
test_df

,0,1,2,3,4
0,82605-69531:Jalen Hurts,82605-29932:Ka'imi Fairbairn,82605-41235:Jake Elliott,82605-71845:Miles Sanders,82605-83074:Dallas Goedert
0,82605-69531:Jalen Hurts,82605-12545:Philadelphia Eagles,82605-29932:Ka'imi Fairbairn,82605-41235:Jake Elliott,82605-72846:A.J. Brown
0,82605-69531:Jalen Hurts,82605-12545:Philadelphia Eagles,82605-29932:Ka'imi Fairbairn,82605-41235:Jake Elliott,82605-71845:Miles Sanders
0,82605-69531:Jalen Hurts,82605-12545:Philadelphia Eagles,82605-33047:Chris Moore,82605-41235:Jake Elliott,82605-72846:A.J. Brown
0,82605-69531:Jalen Hurts,82605-33047:Chris Moore,82605-41235:Jake Elliott,82605-72846:A.J. Brown,82605-90576:DeVonta Smith
...,...,...,...,...,...
0,82605-12545:Philadelphia Eagles,82605-29501:Brandin Cooks,82605-41235:Jake Elliott,82605-86055:Dameon Pierce,82605-90576:DeVonta Smith
0,82605-12545:Philadelphia Eagles,82605-29932:Ka'imi Fairbairn,82605-72846:A.J. Brown,82605-86055:Dameon Pierce,82605-90576:DeVonta Smith
0,82605-12545:Philadelphia Eagles,82605-33047:Chris Moore,82605-71845:Miles Sanders,82605-72846:A.J. Brown,82605-86055:Dameon Pierce
0,82605-12545:Philadelphia Eagles,82605-29501:Brandin Cooks,82605-41235:Jake Elliott,82605-71845:Miles Sanders,82605-89628:Davis Mills


In [81]:
test_df.to_csv('backtest_wk9_thu_phi_hou_predicted_lineups.csv')

In [24]:
if (len(test_df) < 150):
    n = len(test_df)
else:
    n = len(test_df)
test_df.head(n).to_csv('week11_gb_ten_test.csv')